In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

from sklearn.metrics import mean_squared_error as mse
TICK = 'EURUSD'
ANALIZE_TICKERS =['EURUSD', 'USDJPY', 'ICE.BRN', 'comex.GC', 'DJ.A1ENE', 'DJ.E1ENE', 'DJ.A1NCY', 'DJ.E1NCY']
# Стратегия: берем для построения модели ровно три месяца до даты предсказания, т.е если хотим предсказать цены на 22.06.2019
#     периоды должны быть такими:
#     BASE_PER = '_190321_190621'
#     ADD_PER = '_190621_190621'
#     PREDICT_PER = '_190621_190621'

PERIOD_TICK = 1
TESTSIZE = 0.01
BASE_PER = '_190403_190703'
ADD_PER = '_190703_190703'
PREDICT_PER = '_190703_190703'
CONTROL_PER = '_190703_190703'
SEARCH_TARGET = ['model_evaluation', 'prediction']

In [2]:
def del_col(df, cols): #функция удаляет столбцы датафрейма "df", наименование которых переданы в списке "cols"
    for i in cols:
        df = df.drop(i, axis=1)
    return df

In [3]:
def new_df(csv_file): #функция создате новый датафрейм из данных файла "csv_file", преобразуя "наименование" колонок на "наименование+Тикер", также преобразует значение колонки <DATE> ГГГГММЧЧ на ГГГГММДД:ЧЧММСС 
    df = pd.read_csv(csv_file, sep=';')
    ticker = df['<TICKER>'][0]
    df['<DATE>'] = df['<DATE>'].astype(str).str.cat(df['<TIME>'].astype(str), sep=': ')
    df = df.drop('<TIME>', axis=1)
    df = df.drop('<PER>', axis=1)
    df = df.drop('<TICKER>', axis=1)
    df.rename(columns={'<OPEN>':'<OPEN>'+ ticker, '<HIGH>':'<HIGH>'+ ticker,'<LOW>':'<LOW>'+ ticker,'<CLOSE>':'<CLOSE>'+ ticker,'<VOL>':'<VOL>'+ ticker}, inplace=True)
    return df

In [4]:
def merge_df(df_a, df_b): #функция соединяет датафреймы "df_a", "df_b"по колонке <DATE>
    df = pd.merge(df_a, df_b, on='<DATE>', how='inner') #how='outer'
    return df

In [5]:
def merge_all_df(all_df):
    df_all = all_df[0]
    for i in range(1, len(all_df)):
        df_all = merge_df(df_all, all_df[i])
    df_all.fillna(0, inplace=True) #заполняет пустые значения "NAN" на нули "0"
    return df_all
        

In [6]:
def create_x_y_df(df_all, ticker, period=1): #функция разбивающая датафреймы, где df_x и df_y
    df_x = df_all.drop('<DATE>', axis=1)     
    df_y = df_all[['<OPEN>'+ticker, '<HIGH>'+ticker, '<LOW>'+ticker, '<CLOSE>'+ticker, '<VOL>'+ticker]]
    df_x = df_x.iloc[:df_y.shape[0]-period,:]
    df_y = df_y.iloc[period:,:]

    return df_x, df_y

In [7]:
def train_df(x_train, y_train): #функция, строящаяся модель линейной регрессии
    lr_price = lr.fit(x_train, y_train) 
    return lr_price

In [8]:
# функция строящаяся модель предсказание, и предоставляющая датафрейм с предсказанными ценнами на основе цен, содержащихся в df_to_predict
def ticker_predict(df_all, ticker, period = 1, testsize = 0.25, predict = 0, df_to_predict=0):
    x, y = create_x_y_df(df_all, ticker, period)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = testsize, random_state = 42)
    
    if predict == 1:
        x_test = df_to_predict
        lr_open = train_df(x_train, y_train['<OPEN>'+ticker])
        y_pred_open = lr_open.predict(x_test)       
        lr_high = train_df(x_train, y_train['<HIGH>'+ticker])
        y_pred_high = lr_high.predict(x_test)        
        lr_low = train_df(x_train, y_train['<LOW>'+ticker])
        y_pred_low = lr_low.predict(x_test)        
        lr_close = train_df(x_train, y_train['<CLOSE>'+ticker])
        y_pred_close = lr_close.predict(x_test)
        lr_vol = train_df(x_train, y_train['<VOL>'+ticker])
        y_pred_vol = lr_vol.predict(x_test)
        check_test = pd.DataFrame({'Ожидаемые цены y_pred_open':y_pred_open, 'y_pred_high':y_pred_high, 'y_pred_low':y_pred_low, 'y_pred_close':y_pred_close, 'y_pred_vol':y_pred_vol})
        return check_test
    else: 
        lr_open = train_df(x_train, y_train['<OPEN>'+ticker])
        y_pred_open = lr_open.predict(x_test)
        lr_high = train_df(x_train, y_train['<HIGH>'+ticker])
        y_pred_high = lr_high.predict(x_test)
        lr_low = train_df(x_train, y_train['<LOW>'+ticker])
        y_pred_low = lr_low.predict(x_test)
        lr_close = train_df(x_train, y_train['<CLOSE>'+ticker])
        y_pred_close = lr_close.predict(x_test)
        lr_vol = train_df(x_train, y_train['<VOL>'+ticker])
        y_pred_vol = lr_vol.predict(x_test)
        check_test = pd.DataFrame({'y_test_open':y_test['<OPEN>'+ticker], 'y_pred_open':y_pred_open, 'y_test_high':y_test['<HIGH>'+ticker], 'y_pred_high':y_pred_high, 'y_test_low':y_test['<LOW>'+ticker], 'y_pred_low':y_pred_low, 'y_test_close':y_test['<CLOSE>'+ticker], 'y_pred_close':y_pred_close, 'y_test_vol':y_test['<VOL>'+ticker], 'y_pred_vol':y_pred_vol})
        initial_mse_open = mse(y_test['<OPEN>'+ticker], y_pred_open)
        initial_mse_high = mse(y_test['<HIGH>'+ticker], y_pred_high)
        initial_mse_low = mse(y_test['<LOW>'+ticker], y_pred_low)
        initial_mse_close = mse(y_test['<CLOSE>'+ticker], y_pred_close)
        initial_mse_vol = mse(y_test['<VOL>'+ticker], y_pred_vol)
        print(f"mse_open: {initial_mse_open}, mse_high: {initial_mse_high}, mse_low: {initial_mse_low}, mse_close: {initial_mse_close}, mse_vol: {initial_mse_vol}")
        return check_test


In [9]:
def readfiles_csv(files):
    all_df = []
    for i in range(len(files)):
        all_df.append(new_df(files[i]))
    df_all = merge_all_df(all_df)
    return df_all

In [10]:
def concan_df(df_base, add_per):
    files_pred = [] 
    for i in range(len(ANALIZE_TICKERS)): 
        files_pred.append(ANALIZE_TICKERS[i]+add_per+'.csv')
    df_all_pred = readfiles_csv(files_pred) #Датафрейм дня (закончившегося) на основании данных которого идет предсказание цен на следующий день.
    df_all_train = pd.concat([df_base, df_all_pred], axis=0) #Добавление данных закончившегося дня к датафрейму-базе для тренировкиadd_per, построение модели-предсказания
    return df_all_train

In [11]:
def search(search_target=SEARCH_TARGET[1], from_file='Yes', df_concat='Yes', add_per=0):
    if from_file == 'Yes':
        df_all_train = pd.read_csv('df_all'+TICK+'.csv') 
    else: 
        files_train = [] 
        for i in range(len(ANALIZE_TICKERS)):
            files_train.append(ANALIZE_TICKERS[i]+BASE_PER+'.csv')
        df_all_train = readfiles_csv(files_train)
    if df_concat == 'Yes':
        df_all_train = concan_df(df_all_train, add_per)
            
    if search_target == SEARCH_TARGET[0]: #target = model_evolution
        check_test = ticker_predict(df_all_train, TICK, PERIOD_TICK, TESTSIZE, 0, 0) # Предсказание цены на завтра по база df_to_predict      
    elif search_target == SEARCH_TARGET[1]: #target = prediction
        files_pred = [] 
        for i in range(len(ANALIZE_TICKERS)):
            files_pred.append(ANALIZE_TICKERS[i]+PREDICT_PER+'.csv')
        df_all_pred = readfiles_csv(files_pred) #Датафрейм дня (закончившегося) на основании данных которого идет предсказание цен на следующий день.
        df_all_train = pd.concat([df_all_train, df_all_pred], axis=0) #Добавление данных закончившегося дня к датафрейму-базе для тренировки, построение модели-предсказания
        df_to_predict, prev_price = create_x_y_df(df_all_pred, TICK, 0) #Вытаскивание данных из датафрейма (df_all_pred) для отправки в модель-предсказание 
        # df_to_predict #На основании данного датафрейма происходит предсказание цен на следующий день
        check_test = ticker_predict(df_all_train, TICK, PERIOD_TICK, TESTSIZE, 1, df_to_predict) # Предсказание цены на завтра по база df_to_predict
        # 0 - оценка тренированной модели предсказания
        # 1 - боевая модель предсказания на следующий день. Т.е. реальные прогнозы на цены по TICK
    df_all_train.to_csv('df_all'+TICK+'.csv', index=False, encoding='utf-8')  
    return check_test  

#### Построение модели предсказание цены на сделки TICK
по функции 
def search(search_target=SEARCH_TARGET[1], from_file='Yes', df_concat='concat')
где
SEARCH_TARGET = ['model_evaluation', 'prediction'] 
from_file='Yes' - означает считывание существующей базы из файла ('df_all'+TICK+'.csv')
df_concat='concat' - означает что добавляет значения из базы (df_all_pred) к базе 'df_all_train'

In [12]:
check_test = search(SEARCH_TARGET[1], 'No', 'No', ADD_PER) # def search(search_target=SEARCH_TARGET[1], from_file='Yes', df_concat='concat')

In [13]:
check_test.describe() 

,Ожидаемые цены y_pred_open,y_pred_high,y_pred_low,y_pred_close,y_pred_vol
count,1.00000,1.000000,1.000000,1.000000,1.000000e+00
mean,1.12825,1.132624,1.126886,1.128655,1.661658e+06
std,NaN,NaN,NaN,NaN,NaN
min,1.12825,1.132624,1.126886,1.128655,1.661658e+06
25%,1.12825,1.132624,1.126886,1.128655,1.661658e+06
50%,1.12825,1.132624,1.126886,1.128655,1.661658e+06
75%,1.12825,1.132624,1.126886,1.128655,1.661658e+06
max,1.12825,1.132624,1.126886,1.128655,1.661658e+06


In [14]:
check_test

,Ожидаемые цены y_pred_open,y_pred_high,y_pred_low,y_pred_close,y_pred_vol
0,1.12825,1.132624,1.126886,1.128655,1.661658e+06


In [15]:
# Создание базы-контроля, для проверки насколько хорошо были предсказаны цены на текущий день
files_contr = []
for i in range(len(ANALIZE_TICKERS)): 
        files_contr.append(ANALIZE_TICKERS[i]+CONTROL_PER+'.csv')
df_all_contr = readfiles_csv(files_contr)
df_date, contr_price = create_x_y_df(df_all_contr, TICK, 0) # Фактические цены контрольные, по которым сегодня были заключены цены на инструмент
# contr_price  -  реальные цены по сделкам, цены на которых мы прогнозировали 

In [16]:
# contr_price.describe() #фактичекие цены по которым заключались сделки по инструменту TICK
contr_price

,<OPEN>EURUSD,<HIGH>EURUSD,<LOW>EURUSD,<CLOSE>EURUSD,<VOL>EURUSD
0,1.12845,1.13124,1.1267,1.12771,1506695
